In [1]:

from api.websearch.milvus_service import MilvusService
from api.websearch.utils import *

/home/user/ENTER/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/ENTER/envs/langchain/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


ImportError: attempted relative import with no known parent package

In [ ]:
prompt_search = ('<指令>根据你实时联网检索到的信息，更加专业的来回答用户提出的问题。如果无法从中得到答案，请说 “根据检索到的信息无法回答该问题”，'
                 '同时，如果存在历史对话信息，请结合历史对话信息提供完整的回复，'
                 '不允许在答案中添加编造成分，答案请使用中文。 </指令>\n'
                 '<联网检索到的信息>{{ context }}</联网检索到的信息>\n'
                 '<问题>{{ question }}</问题>\n')

prompt_empty = ('请你回答我的问题:\n'
                '{{ question }}\n\n')

In [ ]:
query = "皮卡丘的招式有哪些？"

In [ ]:
# 根据用户输入的问题，调用SerperAPI执行联网检索，返回search_top_k个相关的链接
search_results = await search(query, 10)

# 对检索到的网址链接，通过计算 query 和 每个网站的简介，进一步做 rerank，提取最相关的一个网址
rerank_results = reranking(query, search_results, 8)

# 对经过rerank 的 网站，提取主体内容。
detail_results = await fetch_details(rerank_results)

In [ ]:
rerank_results

In [ ]:
context = " "
if detail_results == []:
    prompt_template = prompt_empty
else:
    # 提取向量数据库的实例,联网检索，统一进入同一个默认向量库中检索
    milvusService = MilvusService(collection_name="test",
                                  overwrite=True,
                                  embedding_model="bge-m3-pro")

    # 添加文档到 milvus 服务
    milvusService.insert_documents(detail_results)
    search_retriever = milvusService.similarity_search(query, k=3, rerank=True)

    context = "\n".join([doc.page_content for doc in search_retriever])

    if len(search_retriever) == 0:  # 如果没有找到相关文档，使用empty模板
        prompt_template = prompt_empty
    else:
        prompt_template = prompt_search

In [ ]:
search_retriever

In [ ]:
context

In [ ]:
rerank_snippets = [doc.metadata['snippet'] for doc in rerank_results if 'snippet' in doc.metadata]

In [ ]:
search_contents = [doc.page_content for doc in search_retriever]

In [ ]:
# 合并并去重
combined_contents = list(set(rerank_snippets + search_contents))

In [ ]:
combined_contents

In [ ]:
# 拼接成最终context
context = "\n\n".join(combined_contents)